<a href="https://colab.research.google.com/github/jferreiraz/BookRecomendation/blob/main/BookRecomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Libray imports

In [ ]:
# import libraries
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

Data files (Unchanged)

In [ ]:
# getting data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2022-12-01 22:45:00--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  27.9MB/s    in 0.9s    

2022-12-01 22:45:01 (27.9 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


CSV import

In [ ]:
# csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

Code

In [ ]:
filter_n1 = df_ratings['user'].value_counts()
filter_n2 = df_ratings['isbn'].value_counts()
df_ratings = df_ratings[~df_ratings['user'].isin(filter_n1[filter_n1 < 200].index) & ~df_ratings['isbn'].isin(filter_n2[filter_n2 < 100].index)]

In [ ]:
df_table = df_ratings.pivot_table(index='isbn', columns='user', values='rating').fillna(0)
df_table

user,11676,16795,21014,23768,35859,43246,52584,55492,60244,76352,...,102967,135149,153662,185233,196077,198711,204864,230522,232131,238120
isbn,,,,,,,,,,,,,,,,,,,,,
0060502258,8.0,0.0,0.0,0.0,7.0,0.0,0.0,5.0,8.0,0.0,...,0.0,0.0,10.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0
0060928336,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,...,0.0,0.0,0.0,0.0,7.0,0.0,9.0,0.0,0.0,0.0
0060930535,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0
0060934417,0.0,0.0,0.0,8.0,9.0,0.0,0.0,0.0,0.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0
0060976845,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
080410526X,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
0804106304,0.0,0.0,0.0,0.0,8.0,0.0,7.0,0.0,8.0,0.0,...,0.0,7.0,9.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0
080410753X,8.0,0.0,0.0,0.0,0.0,7.0,8.0,0.0,0.0,0.0,...,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_table.index = df_table.join(df_books.set_index('isbn'))['title']
df_table

user,11676,16795,21014,23768,35859,43246,52584,55492,60244,76352,...,102967,135149,153662,185233,196077,198711,204864,230522,232131,238120
title,,,,,,,,,,,,,,,,,,,,,
The Divine Secrets of the Ya-Ya Sisterhood: A Novel,8.0,0.0,0.0,0.0,7.0,0.0,0.0,5.0,8.0,0.0,...,0.0,0.0,10.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0
Divine Secrets of the Ya-Ya Sisterhood: A Novel,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,...,0.0,0.0,0.0,0.0,7.0,0.0,9.0,0.0,0.0,0.0
The Poisonwood Bible: A Novel,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0
Bel Canto: A Novel,0.0,0.0,0.0,8.0,9.0,0.0,0.0,0.0,0.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0
Little Altars Everywhere: A Novel,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
All I Really Need to Know,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
The Joy Luck Club,0.0,0.0,0.0,0.0,8.0,0.0,7.0,0.0,8.0,0.0,...,0.0,7.0,9.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0
The Kitchen God's Wife,8.0,0.0,0.0,0.0,0.0,7.0,8.0,0.0,0.0,0.0,...,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Test Base (Unchanged)

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  recommended_books = []
  nbrs = NearestNeighbors(n_neighbors=6, metric="cosine").fit(df_table.values)
  distances, indices = nbrs.kneighbors([df_table.loc[book].values], n_neighbors=6)
  for i in range(1,6):
    recommended_books.append([df_table.index[indices[0][-i]], distances[0][-i]])
  
  return [book, recommended_books]

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['Violets Are Blue', 0.242661], ['The Testament', 0.23662043], ['The Street Lawyer', 0.23472488], ['The Da Vinci Code', 0.22260845], ['Hannibal', 0.17240119]]]
You haven't passed yet. Keep trying!
